In [ ]:
from pathlib import Path
import sys

import polars as pl


sys.path.append(str(Path.cwd().parent))

from libraries.client_stashapp import StashAppClient, get_stashapp_client


stash = get_stashapp_client()
stash_client = StashAppClient()

In [2]:
studio_id = 3

studio_scenes = stash_client.find_scenes_by_studio(studio_id)
studio_galleries = stash_client.find_galleries_by_studio(studio_id)

In [3]:
studio_scenes_with_galleries = studio_scenes.filter(pl.col("stashapp_galleries").list.len() == 1).with_columns(
    pl.col("stashapp_galleries").map_elements(lambda x: x[0]["id"], return_dtype=pl.Int64).alias("id_gallery"),
    pl.col("stashapp_galleries").map_elements(lambda x: x[0]["title"], return_dtype=pl.Utf8).alias("title_gallery"),
)
studio_galleries_with_scenes = studio_galleries.filter(pl.col("stashapp_scenes").list.len() == 1).with_columns(
    pl.col("stashapp_scenes").map_elements(lambda x: x[0]["id"], return_dtype=pl.Int64).alias("id_scene"),
    pl.col("stashapp_scenes").map_elements(lambda x: x[0]["title"], return_dtype=pl.Utf8).alias("title_scene"),
)

In [ ]:
joined_scenes_to_galleries = (
    studio_scenes_with_galleries
    .join(
        studio_galleries_with_scenes,
        left_on="id_gallery",
        right_on="stashapp_id",
        how="left",
        coalesce=False,
        suffix="_gallery",
    )
    .select(
        pl.col("stashapp_id"),
        pl.col("stashapp_title"),
        pl.col("stashapp_date"),
        pl.col("stashapp_performers").map_elements(
            lambda x: [p.get("stashapp_performers_id") for p in x],
            return_dtype=pl.List(pl.Int64),
        ).alias("stashapp_performers_ids"),
        pl.col("stashapp_performers").map_elements(
            lambda x: [p.get("stashapp_performers_name") for p in x],
            return_dtype=pl.List(pl.Utf8),
        ).alias("stashapp_performer_names"),
        pl.col("stashapp_galleries"),
        pl.col("stashapp_id_gallery"),
        pl.col("stashapp_title_gallery"),
        pl.col("stashapp_date_gallery"),
        pl.col("stashapp_performers_gallery").map_elements(
            lambda x: [p.get("stashapp_performers_id") for p in x],
            return_dtype=pl.List(pl.Int64),
        ).alias("stashapp_performers_ids_gallery"),
        pl.col("stashapp_performers_gallery").map_elements(
            lambda x: [p.get("stashapp_performers_name") for p in x],
            return_dtype=pl.List(pl.Utf8),
        ).alias("stashapp_performers_names_gallery"),
        pl.col("stashapp_scenes"),
    )
    .with_columns(
        pl.col("stashapp_performers_ids")
        .list.set_difference(pl.col("stashapp_performers_ids_gallery"))
        .alias("performers_to_add"),
    )
    .filter(
        pl.col("stashapp_id_gallery").is_not_null(),
        pl.col("performers_to_add").list.len() > 0,
    )
)
current_batch = joined_scenes_to_galleries.head(10)
current_batch

In [ ]:
current_batch_update_galleries_inputs = []
for row in current_batch.iter_rows(named=True):
    current_batch_update_galleries_inputs.append({
        "id": row["stashapp_id_gallery"],
        "performer_ids": row["stashapp_performers_ids"],
    })
current_batch_update_galleries_inputs


In [35]:
for update_gallery_input in current_batch_update_galleries_inputs:
    stash.update_gallery(update_gallery_input)